In [1]:
from autogen import ConversableAgent
from openai import OpenAI
from pathlib import Path

import LLM_configs
import Prompt_dics
from LLM_configs import *
from Agents import *
from tools import *

In [2]:
# 加载 .env 文件
load_dotenv()

True

## 文件提取

In [3]:
text = file_extraction("test files/巨量均衡 2.docx")

In [4]:
PPT = file_extraction("test files/上海象上投资.pptx")

## Agent设计

In [59]:
quant_dict = {
    'quant_idea':[],
    'factor_composition':[],
    'factor_explanation':[],
    'model_explanation':[],
    'factor_update':[],
    'strategy_execution':[]
}

In [60]:
market_neutral = fill_placeholders(['中证500指增', '股指期货','0%'], Prompt_dics.Other_prompt_dic['中性多头/对冲端'])

In [ ]:
market_neutral

In [53]:
def report_writing(weak_config, strong_config,prompts,component_dict, requirements, updating):
    extraction_agent = extraction_assistant(weak_config)
    
    integrate_agent = integrate_assistant(strong_config,requirements)
    
    for prompt, component in zip(prompts, list(component_dict.keys())):
        reply = extraction_agent.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": prompt, "role": "user"}])
        component_dict[component].append(reply)
    
    component_dict['hedging tools'] = market_neutral
    
    final_report = integrate_agent.generate_reply(messages=[{"content": str(component_dict), "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    non_updating_report = final_report
    
    if updating:
        old_report_relevant_part = input("请把上一篇报告的配置逻辑部分复制到此处")
        final_report = integrate_agent.generate_reply(messages=[{"content": "旧报告: "+old_report_relevant_part+"新报告:"+str(final_report), "role": "system"},
                                                                {"content": "你是一个报告审查和撰写专员，现在你有一篇旧报告和一篇新报告，你要做的是在旧报告的基础上，判断新报告中的增量信息，并将新报告中的增量信息与旧报告中的现有信息相结合，完成一篇更新的报告，如果两个报告中对某一个地方有冲突，则以新报告为准。", "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整.长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    
    cost_list = [extraction_agent.get_total_usage(),integrate_agent.get_total_usage()]
    
    return final_report,non_updating_report, component_dict, cost_list

In [54]:
Ali_report,first_Ali_report, Quant_component_dic, cost_list = report_writing(LLM_configs.Qwen_weak_llm_config, LLM_configs.Qwen_strong_llm_config, Prompt_dics.Strategy_prompt_dic['Simple Quant Strategy'],quant_dict, Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy'],True)


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


In [55]:
print(Ali_report)

管理人的量化策略整体构建在逻辑驱动的基础之上，特别强调因子的可解释性，以此为核心思路。相较于旧报告中提及的线性模型与机器学习模型并重的策略，新报告揭示了管理人在因子组成上的新布局：60%的策略权重分配给了基本面量化，剩余40%则归于量价因子，这一调整体现了对优质公司识别逻辑的进一步强化及对市场交易行为捕捉的精炼整合。因子总数量维持在5000至6000个的宽广范围内，但实际每期应用的因子被精心筛选至数百个，确保了因子的有效性和针对性。

在因子挖掘方法上，公司投入超过60%的精力于因子研究，采用“先逻辑后量化”的独特路径，这与旧报告中提及的人工与机器学习混合的方法形成对比，更加凸显了逻辑思维在因子构建初期的关键作用。尽管如此，量价因子的构建仍融入了机器学习技术，如神经网络，显示了在特定领域的技术深化。

因子处理方面，基本面因子主要通过线性及简单非线性模型进行合成与处理，确保逻辑清晰与结果的可解释性，而量价因子则运用了更为复杂的模型组合，包括GRU、RNN等，旨在捕捉市场的复杂动态。这种差异化处理策略，既保留了策略的逻辑性，又增强了对市场非线性特征的适应能力。

量化模型构建上，公司坚持“重因子轻模型”的原则，更重视因子的精选而非模型的复杂度，这与旧报告中提及的模型参数调整频率不高、重视模型稳定性的理念相呼应。模型应用涵盖了线性、非线性模型及机器学习模型，如决策树，展现了策略的灵活性与多样性。

新报告中未直接阐述因子入库的具体条件，但基于旧报告的背景，可以推测管理人对新因子的引入同样遵循严格的标准和流程，包括模拟盘测试、风险评估等环节。

在策略执行层面，管理人展现了高度的市场敏感性和策略调整能力，如在极端市场情况下减仓并在市场恢复时迅速调整策略回归。策略的实施细节虽未全面展开，但提及了策略容量的扩展潜力，基本面策略的容量预估可达百亿级别，且存在行业轮动模型指导行业配置，尽管具体偏离度未明示。交易执行方面，未直接提及算法下单的使用情况，但考虑到管理人的技术倾向，可能已采用或计划采用先进的交易算法以优化执行效率。

对冲策略方面，新报告明确了管理人采用股指期货对冲多头端的中证500指增策略，不留风险敞口，这与旧报告中未明确的对冲方法相比，提供了更具体的对冲实践信息。

综上所述，管理人的量化策略在保持原有核心逻辑的同时，对因子构成、挖掘方法、处理方式、模型选择及对冲策

In [56]:
print(first_Ali_report)

该公司的量化策略整体上偏重逻辑驱动，对因子的可解释性有着极高的要求。在因子构建方面，基本面因子占据60%的比重，这部分因子构建非常注重逻辑性，强调对优质公司的识别；量价因子占40%的比重，这部分虽然侧重于捕捉市场交易行为，但也并非纯粹的数据挖掘，而是结合了一定的逻辑思考。具体来说，基本面因子中，财务指标和公告相关因子占40%，分析师数据同样占40%，另类数据（如产业链上下游数据、舆情数据）占20%。因子池总数约为5000到6000个，但每期实际使用的因子数量大约为几百个。

在因子挖掘方法上，公司大约花费60%-70%的精力在因子研究上，而模型构建则相对简单，约20%的精力用于优化器。因子覆盖范围广泛，包括基本面因子、量价因子和另类因子等，其中基本面因子的研究尤其深入。在基本面因子方面，公司采取“先逻辑后量化”的方式，即先确定投资逻辑，再通过量化手段表达这一逻辑。这意味着公司不是简单地通过数据挖掘寻找因子，而是基于深刻的逻辑思考来构建因子。相对于基本面因子，量价因子的构建则可能使用更为复杂的模型和技术，包括机器学习方法，但这部分在整体策略中的权重较低。

在因子处理方法上，对于基本面因子，主要采用线性模型和简单的非线性模型，这表明在这一领域更重视逻辑性和可解释性；量价因子的应用则更为多样化，不仅包括传统的线性模型，还包括一些复杂的机器学习模型，如神经网络等，这反映了在捕捉市场非理性交易行为时对模型复杂度的需求。

在量化模型方面，公司倾向于“重因子轻模型”的策略，即更看重因子的质量而非模型的复杂度。在基本面因子的应用上，主要采用线性模型和简单的非线性模型，在量价因子的应用上则使用了更为复杂的模型，包括但不限于线性模型、非线性模型（如决策树）、GRU（门控循环单元）和RNN（循环神经网络）等。

在因子入库条件方面，根据提供的内容，没有明确提到管理人的新因子入库的具体条件，包括在模拟盘中观察的时间、衡量指标如夏普比率、ICIR等以及其他措施。

在具体执行方面，管理人进行了策略调整，年前最后一周之前，面对极端扩大的基差，进行了一定程度的减仓；在2月8日开盘后，通过市场监测特别是ETF的放量情况，进行了策略的调整回归。策略特点上，因子模型构建上重因子轻模型，基本面因子约占60%，量价因子约占40%。现行策略的换手情况未明确提及，但提到了策略的容量规模大概在15亿左右，未

In [57]:
cost_list

[{'total_cost': 0.26046800000000003,
  'qwen-plus': {'cost': 0.26046800000000003,
   'prompt_tokens': 60341,
   'completion_tokens': 1592,
   'total_tokens': 61933}},
 {'total_cost': 0.030975999999999997,
  'qwen-plus': {'cost': 0.030975999999999997,
   'prompt_tokens': 5695,
   'completion_tokens': 683,
   'total_tokens': 6378}}]

In [ ]:
cost_list

In [58]:
Quant_component_dic

{'策略思路': ['该公司的量化策略整体上是偏重逻辑的，对因子的可解释性有很高的要求。具体体现在以下几个方面：\n\n1. **因子构建**：\n   - 基本面因子占60%的比重，这部分因子构建非常注重逻辑性，强调对优质公司的识别。\n   - 量价因子占40%的比重，这部分虽然相对更侧重于捕捉市场交易行为，但也并非纯粹的数据挖掘，而是结合了一定的逻辑思考。\n\n2. **因子来源**：\n   - 财务指标和公告相关因子占基本面因子的40%，这部分数据直接来源于上市公司的财报等公开信息。\n   - 分析师数据同样占基本面因子的40%，这部分数据反映了市场专业人士的观点。\n   - 另类数据（如产业链上下游数据、舆情数据）占20%，这部分数据有助于从不同角度理解市场情绪和行业趋势。\n\n3. **模型应用**：\n   - 在基本面因子的应用上，主要采用线性模型和简单的非线性模型，这表明在这一领域更重视逻辑性和可解释性。\n   - 量价因子的应用则更为多样化，不仅包括传统的线性模型，还包括一些复杂的机器学习模型，如神经网络等，这反映了在捕捉市场非理性交易行为时对模型复杂度的需求。\n\n4. **策略特点**：\n   - 整体策略倾向于“重因子轻模型”，即更看重因子的质量而非模型的复杂度。\n   - 因子池中有5000-6000个因子，但实际使用的因子数量大约为几百个，这表明在众多因子中进行了严格的筛选，以确保因子的有效性和逻辑性。\n\n综上所述，该公司的量化策略在构建因子和选择模型时都强调逻辑性和可解释性，而不是单纯依赖数据挖掘或复杂的机器学习算法。',
  '该公司的量化策略整体上是偏重逻辑的，对因子的可解释性有很高的要求。具体体现在以下几个方面：\n\n- 在因子模型构建上，公司大约花费60%-70%的精力在因子研究上，而模型构建则相对简单，主要依赖线性模型和简单的非线性模型。\n- 因子研究覆盖广泛，包括基本面因子、量价因子和另类因子等，其中基本面因子的研究尤其深入。\n- 基本面因子研究中，40%的因子来源于财务指标和公告信息，40%来源于分析师数据，20%来源于另类数据（如产业链上下游数据和舆情数据）。\n- 量价因子研究则侧重于捕捉非理性交易机会，分为高频、中期和长期量价因子，并且会将基本面信息融入量价因子中。\n\n整体而言，该公

In [ ]:
def report_updating(weak_config, strong_config,prompts,component_dict, requirements):
    